In [1]:
import pandas as pd
from tqdm import tqdm

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("senylar/sis-text-class")

print("Path to dataset files:", path)

In [2]:
import re

def preprocess(text):
    # Превращаем в нижний регистр
    text = text.lower()
    # Удаляем всё, кроме букв и пробелов
    text = re.sub(r'[^\w\s]', '', text)
    # Разбиваем на слова
    tokens = text.split()
    return tokens

print(preprocess("Очень хороший сервис! Приду ещё."))
# ['очень', 'хороший', 'сервис', 'приду', 'ещё']


['очень', 'хороший', 'сервис', 'приду', 'ещё']


In [3]:
train = pd.read_csv('train.csv')

In [4]:
texts = train['text']
train_sample = train.sample(n=50000, random_state=42)
texts = train_sample['text'].tolist()
labels = train_sample['sentiment'].tolist()

tokenized_texts = [preprocess(t) for t in texts]

vocab = {"<PAD>": 0, "<UNK>": 1}
for tokens in tokenized_texts:
    for token in tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=50):
        self.vocab = vocab
        self.max_len = max_len
        self.label_map = {"negative": 2, "neutral": 0, "positive": 1}

        self.tokenized = [self.encode(preprocess(t)) for t in texts]
        self.labels = labels
    def encode(self, tokens):
        ids = [self.vocab.get(t, self.vocab["<UNK>"]) for t in tokens]
        return ids[:self.max_len] + [self.vocab["<PAD>"]] * (self.max_len - len(ids))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.tokenized[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)


batch_size = 32

train_dataset = SentimentDataset(texts, labels, vocab)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [16]:
class SentimentModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)  # вот тут
        self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)            # [B, T] -> [B, T, E]
        _, h = self.rnn(x)               # [B, T, E] -> [1, B, H]
        out = self.fc(h.squeeze(0))      # [B, H] -> [B, C]
        return out

# Инициализация
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentimentModel(vocab_size=len(vocab), embedding_dim=250, hidden_dim=64, num_classes=3)
model.to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.003)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

In [17]:
# Тренировка
for epoch in tqdm(range(4)):
  for X_batch, y_batch in train_loader:
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)

    model.train()
    optimizer.zero_grad()
    out = model(X_batch)
    loss = loss_fn(out, y_batch)
    loss.backward()
    optimizer.step()


  preds = torch.argmax(out, dim=1)
  acc = (preds == y_batch).float().mean()
  print(f"Эпоха {epoch}, loss={loss.item():.4f}, accuracy={acc:.2f}")


  7%|▋         | 1/15 [00:45<10:38, 45.62s/it]

Эпоха 0, loss=0.6108, accuracy=0.62


 13%|█▎        | 2/15 [01:31<09:53, 45.63s/it]

Эпоха 1, loss=0.4702, accuracy=0.88


 20%|██        | 3/15 [02:16<09:07, 45.63s/it]

Эпоха 2, loss=0.4279, accuracy=0.88


 27%|██▋       | 4/15 [03:02<08:21, 45.64s/it]

Эпоха 3, loss=0.0487, accuracy=1.00


 27%|██▋       | 4/15 [03:16<09:01, 49.23s/it]


KeyboardInterrupt: 

In [18]:
torch.save(model.state_dict(), "sentiment_model.pth")

In [22]:
def encode(tokens, vocab, max_len=120):
    ids = [vocab.get(t, vocab["<UNK>"]) for t in tokens]
    if len(ids) < max_len:
        ids += [vocab["<PAD>"]] * (max_len - len(ids))
    else:
        ids = ids[:max_len]
    return ids


def predict(text, model, vocab):
    model.eval()  # переводим в режим оценки
    tokens = preprocess(text)                     # Токенизация

    encoded = encode(tokens, vocab)               # Преобразуем в индексы
    tensor = torch.tensor([encoded]).to(device)  # Преобразуем в тензор и на нужное устройство

    with torch.no_grad():
        output = model(tensor)                    # Получаем логиты
        pred = torch.argmax(output, dim=1).item() # Выбираем класс с максимальной вероятностью

    # Словарь отображения метки в строку
    idx2label = {0: "neutral", 1: "positive", 2: "negative"}
    return idx2label[pred]


# --- 7. Ввод с клавиатуры ---
for _ in range(3):
    user_text = input("Введите текст (или 'выход' для завершения): ")
    if user_text.strip().lower() == "выход":
        break
    print("Оценка:", predict(user_text, model, vocab))

Введите текст (или 'выход' для завершения): Самый лучший отель в жизни
Оценка: positive
Введите текст (или 'выход' для завершения): Этот отель - это самый ужасный и худший из всех
Оценка: negative
Введите текст (или 'выход' для завершения): Нормас отель
Оценка: neutral
